# LangGraph로 만드는 AI 에이전트 실전 입문

## 9.3 Q&A 애플리케이션 실습

In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [ ]:
ROLES = {
    "1": {
        "name": "일반 지식 전문가",
        "description": "폭넓은 분야의 일반적인 질문에 답변",
        "details": "폭넓은 분야의 일반적인 질문에 대해 정확하고 이해하기 쉬운 답변을 제공해 주세요."
    },
    "2": {
        "name": "생성 AI 제품 전문가",
        "description": "생성 AI와 관련 제품, 기술에 관한 전문적인 질문에 답변",
        "details": "생성 AI와 관련 제품, 기술에 관한 전문적인 질문에 대해 최신 정보와 깊은 통찰력을 제공해 주세요."
    },
    "3": {
        "name": "카운슬러",
        "description": "개인적인 고민이나 심리적인 문제에 대해 지원 제공",
        "details": "개인적인 고민이나 심리적인 문제에 대해 공감적이고 지원적인 답변을 제공하고, 가능하다면 적절한 조언도 해주세요."
    }
}

In [ ]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="사용자의 질문")
    current_role: str = Field(
        default="", description="선정된 답변 역할"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="답변 이력"
    )
    current_judge: bool = Field(
        default=False, description="품질 체크 결과"
    )
    judgement_reason: str = Field(
        default="", description="품질 체크 판단 이유"
    )

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 나중에 max_tokens 값을 변경할 수 있도록 변경 가능한 필드 선언
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [ ]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""질문을 분석하고 가장 적합한 답변 담당 역할을 선택해 주세요.

선택지:
{role_options}

답변은 선택지의 번호(1, 2, 또는 3)만 반환해 주세요.

질문: {query}
""".strip()
    )
    # 선택지의 번호만 반환되기를 기대하므로 max_tokens 값을 1로 변경
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [ ]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""당신은 {role}로서 답변해 주세요. 다음 질문에 대해 당신의 역할에 기반한 적절한 답변을 제공해 주세요.

역할의 상세 내용::
{role_details}

질문: {query}

답변:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [ ]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="판정 결과")
    reason: str = Field(default="", description="판정 이유")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""다음 답변의 품질을 체크하고, 문제가 있으면 'False', 문제가 없으면 'True'로 답변해 주세요.
또한, 그 판단 이유도 설명해 주세요.

사용자의 질문: {query}
답변: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [ ]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [ ]:
# selection 노드에서 처리 시작
workflow.set_entry_point("selection")

In [ ]:
# selection 노드에서 answering 노드로
workflow.add_edge("selection", "answering")
# answering 노드에서 check 노드로
workflow.add_edge("answering", "check")

In [ ]:
from langgraph.graph import END

# check 노드에서 다음 노드로의 전환에 조건부 엣지 정의
# state.current_judge 값이 True면 END 노드로, False면 selection 노드로
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [ ]:
compiled = workflow.compile()

In [ ]:
initial_state = State(query="생성 AI에 관해 알려주세요")
result = compiled.invoke(initial_state)

In [ ]:
result

In [ ]:
print(result["messages"][-1])

In [ ]:
initial_state = State(query="생성 AI에 관해 알려주세요")
result = await compiled.ainvoke(initial_state)
result

In [ ]:
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
from IPython.display import Image

Image(compiled.get_graph().draw_png())

## 9.4 체크포인트 기능: 상태의 영속화와 재개

In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 langgraph-checkpoint==1.0.11

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [ ]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 그래프의 상태 정의
class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# 메시지를 추가하는 노드 함수
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="당신은 최소한의 응답을 하는 대화 에이전트입니다.")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

# LLM에서의 응답을 추가하는 노드 함수
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [ ]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("체크포인트 데이터:")
    pprint(checkpoint_tuple.checkpoint)
    print("\n메타데이터:")
    pprint(checkpoint_tuple.metadata)

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 그래프 설정
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# 체크포인터 설정
checkpointer = MemorySaver()

# 그래프 컴파일
compiled_graph = graph.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="제가 좋아하는 것은 쿠키앤크림 아이스크림입니다. 기억해 주세요.")
first_response = compiled_graph.invoke(user_query, config)
first_response

In [ ]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

In [ ]:
print_checkpoint_dump(checkpointer, config)

In [ ]:
user_query = State(query="제가 좋아하는 것이 뭔지 기억나세요?")
second_response = compiled_graph.invoke(user_query, config)
second_response

In [ ]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

In [ ]:
print_checkpoint_dump(checkpointer, config)

In [ ]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="제가 좋아하는 것은 뭔가요?")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response